# Integração numérica de funções de uma variável

Jeanlex Soares de Sousa (jeanlex@fisica.ufc.br) <br>
Departamento de Física <br>
Universidade Federal do Ceará


## Introdução

Em análise numérica, a integração numérica constitui um amplo conjunto de algorítmos para calcular o valor numérico de uma integral definida. Por extensão, este termo também é usado para descrever a solução numérica de equaçoes diferenciais. 

$$\int_a^{b} f(x)~dx = c $$

Assuma por exemplo, a função $f(x) = 3 x^2 - x + 2$. A integral desta solução entre um intervalo $[a,b]$ possui solução analítica simples:

$$\int_a^{b} (3 x^2 - x +1)~dx = \left(b^3 - \frac{b^2}{2} +2 b\right) - \left( a^3 - \frac{a^2}{2} + 2 a \right) $$

De forma simplificada, uma integral nos fornece a área abaixo de uma função em um intervalo específico. Veja o coódigo para representar a itegral desta função abaixo:

In [1]:
# Carregando bibliotecas numérica e gráfica
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon

# Carrega a biblioteca de widgets
from ipywidgets import interact
import ipywidgets as widgets

In [2]:
# -----------------------------------------
# definição da função a ser integrada
# -----------------------------------------
def f(x):
    return 3 * (x ** 2) - x + 2

# -----------------------------------------
# função para cálculo analítico da integral
# -----------------------------------------
def calc_integral(a, b):
    calc_integral = (b**3 - 0.5*(b**2) + 2*b) - (a**3 - 0.5*(a**2) + 2*a)
    return calc_integral

# -----------------------------------------
# definição de função para plotagem do gráfico
# -----------------------------------------
def plot_integ_definition(a,b):

    # Define intervalo e número de pontos de plotagem
    x1, x2 = 0, 8
    N = 50

    # cria lista com abscissa e ordenada
    x = np.linspace(x1,x2,N)
    y = f(x)

    # Plotagem da função
    fig, ax = plt.subplots()
    plt.plot(x, y, 'b', linewidth=2)

    # marca área embaixo da função
    ix = np.linspace(a, b, N)
    iy = f(ix)
    verts = [(a, 0)] + list(zip(ix, iy)) + [(b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

    # plota equação no gráfico
    plt.text(0.5 * (a + b), 120, 
         "$\int_a^b f(x)\mathrm{d}x$",
         horizontalalignment='center', fontsize=20)

    plt.show()
    
    print("Integral = %5.3f" % (calc_integral(a,b)))



In [3]:
# -----------------------------------------    
# widget de demonstração
# -----------------------------------------
interact(plot_integ_definition, 
             a=widgets.FloatSlider(min=0,max=3,step=0.25,value=2.0,
                                description = '$a$ = '),
             b=widgets.FloatSlider(min=3,max=8,step=0.25,value=5.0,
                                description = '$b$ = ')
        );        


interactive(children=(FloatSlider(value=2.0, description='$a$ = ', max=3.0, step=0.25), FloatSlider(value=5.0, description='$b$ = ', max=8.0, min=3.0, step=0.25), Output()), _dom_classes=('widget-interact',))

## Integração numérica

Numericamente, a natureza discreta das funções exige que façamos algumas aproximações para conseguirmos calcular a integral. Uma possibilidade é assumir que cada ponto do gráfico seja considerado um retângulo de largura $\Delta x$. Veja abaixo de duas formas de considerar a área abaixo de cada ponto:

In [4]:
# -----------------------------------------
# definição de função para plotagem do gráfico
# -----------------------------------------

def plot_numinteg_def(N):

    # Define intervalo e número de pontos de plotagem
    x1, x2 = 0, 5

    # cria lista com abscissa e ordenada
    x = np.linspace(x1,x2,N)
    y = f(x)
    dx = x[1]-x[0]
    
    # Plotagem da função
    fig, ax = plt.subplots(1,2,figsize=(11, 4))
    ax[0].plot(x, y, 'bo-', linewidth=2)
    for i in range(0,N):
        verts = [(x[i]-dx/2, 0)] + [(x[i]-dx/2,y[i])] + [(x[i]+dx/2,y[i])] + [(x[i]+dx/2, 0)]
        poly = Polygon(verts, facecolor='r', edgecolor='k')
        ax[0].add_patch(poly)
    
    ax[0].set_title('retângulo')
    
    ax[1].plot(x, y, 'bo-', linewidth=2)
    
    for i in range(0,N-1):
        verts = [(x[i], 0)] + [(x[i],y[i])] + [(x[i+1],y[i+1])] + [(x[i+1], 0)]
        poly = Polygon(verts, facecolor='r', edgecolor='k')
        ax[1].add_patch(poly)
        
    ax[1].set_title('trapézio')
    
    plt.show()
    
# -----------------------------------------    
# widget de demonstração
# -----------------------------------------

interact(plot_numinteg_def, 
             N=widgets.IntSlider(min=10,max=40,step=5,value=20,
                                description = '$N$ = ')
        );      
    

interactive(children=(IntSlider(value=20, description='$N$ = ', max=40, min=10, step=5), Output()), _dom_classes=('widget-interact',))

Nas demonstrações acima, é perceptível que o erro diminui à medida que o número $N$ de pontos de plotagem aumenta. Mas também é possível observar que a segunda aproximação fornece um resultado muito melhor que o primeiro mesmo para pequenos valores de $N$. Esta segunda aproximação é conhecida como a **regra do trapézio de Simpson** e constitui o método mais comum de integração numérica. Existem muitos outros métodos de integração numérica, mas para este curso básico, o método de Simpson é suficiente.

Considerando-se as formas acima, a área abaixo das curvas podem ser calculadas usando-se:

$$ I = \sum_{i=1}^N f(x_i) \Delta x $$

ou

$$ I = \sum_{i=1}^N f(x_i) \Delta x - \frac{\Delta_x}{2}\left[f(x_1)+f(x_N)\right] $$

ou 

$$I = \sum_{i=1}^{N-1} \frac{f(x_i)+f(x_{i+1})}{2} \Delta x $$



O código para calcular uma integral numérica é bastante simples:

In [5]:
# -----------------------------------------
# função para realizar integração numérica
# -----------------------------------------

def integral(x,y,tipo=1):
    # esta função serve para dx fixo

    # determina o número N de pontos na função
    N = len(x)
    
    # determina dx
    dx = x[1] - x[0]
    
    if tipo==1:
        
        tmp = 0
        for i in range(0,N):
            tmp = tmp + y[i]*dx

    elif tipo==2:
        
        tmp = 0
        for i in range(0,N):
            tmp = tmp + y[i]*dx
        
        tmp = tmp - (y[0]+y[N-1])*dx/2
    
    else:
        
        tmp = 0
        for i in range(0,N-1):
            tmp = tmp + (y[i]+y[i+1])*dx/2
            
    
    return tmp
    


## Exemplo: $f(x) = 3 x^2 - x + 2$

Compare a abaixo as integrais exata e numérica usando diferentes valores de N usando os métodos descritos acima:

In [16]:
# Define intervalo e número de pontos de plotagem
x1, x2 = 0, 5

# calculo exato da integral
int_exata = calc_integral(x1,x2)

for i in [10,50,100]:
    
    print('')
    print('N = %d' % i)
    
    for j in [1,2,3]:
        
        x = np.linspace(x1,x2,i)
        y = f(x)
        int_num = integral(x,y,j)
        erro = np.abs(int_num-int_exata)
        
        print('Método %d: Integral = %5.3f, Erro = %5.3f' % (j,int_num,erro))
        

    



N = 10
Método 1: Integral = 143.827, Erro = 21.327
Método 2: Integral = 123.272, Erro = 0.772
Método 3: Integral = 123.272, Erro = 0.772

N = 50
Método 1: Integral = 126.302, Erro = 3.802
Método 2: Integral = 122.526, Erro = 0.026
Método 3: Integral = 122.526, Erro = 0.026

N = 100
Método 1: Integral = 124.375, Erro = 1.875
Método 2: Integral = 122.506, Erro = 0.006
Método 3: Integral = 122.506, Erro = 0.006
